# A Conjecture About Means of Prime Powers

Here's a conjecture:

Consider the set of all powers of a prime, from 1 to p^k: P = {p^0, p^1, p^2, ... p^k}.
Pull out two subsets, S & T, that don't intersect.
Take the means of each.

Conjecture: $\mu(S) != \mu(T)$

For example, if p is 2 and k is 10 P = {1, 2, 4, 8, ..., 1024} 
If S = {1, 2, 8} and T = {4, 16, 32}, then $\mu(S) = 11/3$ and $\mu(T) = 52/3$

The conjecture is that this holdsfor any two finite, non-intersecting set of powers of p, a prime. 

Sticking with powers of $p = 2$, here's another way to say this.

First, *every* integer is a unique sum of powers of two.

17 = 16 + 1
 7 = 4 + 2 + 1
11 = 8 + 2 + 1
52 = 32 + 16 + 4

and so on.

You can read that sum right from the number's binary representation.

17 = 10001
7 = 111
11 = 1011
52 = 110100
...

So, the average of a series that sums to N is just N/(# of 1-bits in the binary representation of N) . 

Next, I'll make up a term: I'll call the inverse of this -- (# of 1-bits in the binary representation /  N) -- the 1's density. I'm not crazy about this term and would love a better one. but let me use it for a few minutes.

It does sort of correspond to your intuition. If a number is big but it doesn't have very many 1-bits in its binary representation, then the 1's density is low. If a number's little, but its binary representation is full of 1's, then the 1's density is high.

For example,

$512 = 1000000000(base2)$ -- low 1's density
$15 = 1111(base 2)$ -- high 1's density.

If the averages are the same, the inverses of the averages are the same, too. I'm claiming two different integers won't ever have the same 1's density -- I'm saying 17 (10001) and 8 (1000) won't have the same 1's density.

No, wait. That's not right. I'm claiming that as long as they don't have any of the *same* one bits set, they won't have the same 1's density.

What about a more grandiose question: Will *any* two integers ever have the same 1's density?
Why yes, they will.

When I look at, say, the first 100 integers, 69 and 92 have the same 1's density. :-(
69 has 3 one-bits, 92 has 4 one-bits. (3/69= 1/23 = 4/92).  But they share bits!
69 = 1000101(base 2) and 92 = 1011100(base 2), so they both share 64 = 2^6 and 4 = 2^2.

Here's the conjecture in pseudo-code, for $p = 2$

```
if m != n and !share_bits(m, n) then ones_density(m) != ones_density(n).
```

Here's a shorter way to say that.

```
if ones_density(m) == ones_density(n) then share_bits(m, n)
```

Time to write some code to look for a counterexample.


Let's begin with a couple of functions.

In [ ]:
def ones_density(n):
    """Number of one-bits divided by the number itself."""
    return (n).bit_count()/n

ones_density(3)

In [ ]:
def share_bits(m, n):
    """True iff m and n share a bit."""
    return bool(m&n)
print(share_bits(1, 3))
print(share_bits(8, 4))

Next, collect the `ones_density` of all the numbers from 1 to lim-1.

In [ ]:
def densities(lim):
    """Densities of all numbers from 1 up to but not including lim."""
    d = {}
    for n in range(1, lim):
        d[n] = ones_density(n)
    return d

densities(5)

How many densities are there in all the numbers from 1 up to (but not including) 99?

In [ ]:
d_100 = densities(100)
len(set(d_100.values()))

If every integer had a different density, there would have been 99 values. There are only 98, so some value is there twice -- there's a collision. Where is it?

Let's start out by reversing the mapping -- make a dictionary with the integer as the value and its ones density as its key. Then, every time we look at a new integer, we can see whether we've already seen an integer with that ones value.

If we have, then we report the collision.

In [ ]:
def map_densities_to_ints(lim):
    int_val = {}
    k = int_val.keys()
    for n in range(1, lim):
        d = ones_density(n)
        if d in k:
            print(f"collision: {int_val[d]}, {n}") # the current n, and the first integer with the same ones_density
        else:
            int_val[d] = n
    return int_val

iv_1000 = map_densities_to_ints(200)

Plenty of collisions! For each collision, do colliders share bits?


Let's squirrel away the collisions, then go back and look.

In [ ]:
from collections import defaultdict
def find_collisions(lim):
    int_val = {}
    collisions = defaultdict(set)
    k = int_val.keys()
    for n in range(1, lim):
        d = ones_density(n)
        if d in k:
            c = {n, int_val[d]} # set of the colliding pair
            collisions[d] |= c  # union with the existing set of collisions for that ones_density
        else:
            int_val[d] = n
    return collisions

find_collisions(100)

The conjecture is that collisions are only possible for numbers that have one or more bits in common.
Does this pair share any bits?

In [ ]:
share_bits(69, 92)

Oh? And what bits would those be?

In [ ]:
def show_collision(i, j):
    print(f"{(i).bit_count()}/{i} == {ones_density(i)} == {(j).bit_count()}/{j}")
    print(f"collision: {i}({i:b}) & {j}({j:b}) == {i&j:b}")

show_collision(69, 92)

That's one case. We need a function that will look at all pairs from each collision set.

In [ ]:
def collisions_share_bits(s):
    l = list(s)   # turn it into a list
    while l:
        end = l.pop()   # pop off the rightmost element
        pairwise_shares = [share_bits(end, elem) for elem in l] # check whether all remaining share bits with end
        if not all(pairwise_shares):
            return False
    return True
print(collisions_share_bits({69, 92}))
print(collisions_share_bits({4, 8}))
print(collisions_share_bits({69, 4, 92, 8}))

In [ ]:
def counterexamples(n):
    true_collisions = {}
    for ones_density, collisions in find_collisions(n).items():
        if not collisions_share_bits(collisions):   # Found a counterexample
            true_collisions[ones_density] = collisions
    return true_collisions

counterexamples(10_000)

In [ ]:
show_collision(7554, 8813)

Same ones-density, no shared bits. The conjecture's false. Bummer.